In [2]:
import os

# Define the path to the Google Fit data
google_fit_path = "C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit"

# List all files in the Google Fit directory
for root, dirs, files in os.walk(google_fit_path):
    for file in files:
        print(os.path.join(root, file))


C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit\Activities\2016-05-02T18_27_18.775+05_30_PT2H4M49.792S_Walking.tcx
C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit\Activities\2016-05-03T18_24_20.055+05_30_PT23M4.112S_Walking.tcx
C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit\Activities\2016-05-03T20_23_39.916+05_30_PT20M49.109S_Walking.tcx
C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit\Activities\2016-05-04T18_18_24.187+05_30_PT24M19.948S_Walking.tcx
C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit\Activities\2016-05-04T19_55_29.313+05_30_PT22M15.311S_Walking.tcx
C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit\Activities\2017-02-01T17_59_06.489+05_30_PT1H2M5.856S_Walking.tcx
C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit\Activities\2017-02-03T18_07_04.383+05_30_PT1H10M2.595S_Walking.tcx
C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit\Activities\2017-02-04T17_53_13.82+05_30_PT1H2M15.973S_Walking.tcx
C:/Users/pavan/

In [14]:
# Function to read JSON files
def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# Read all JSON files in the directory
def read_all_json_files(directory):
    json_files = glob(os.path.join(directory, '*.json'))
    all_data = []
    for json_file in json_files:
        data = read_json_file(json_file)
        all_data.extend(data)
    return all_data

# Example: Reading JSON data from "All Data" directory
json_data_all_data = read_all_json_files(all_data_path)
json_data_all_sessions = read_all_json_files(all_sessions_path)

print(f"Total records from 'All Data': {len(json_data_all_data)}")
print(f"Total records from 'All Sessions': {len(json_data_all_sessions)}")


Total records from 'All Data': 280
Total records from 'All Sessions': 4050


In [16]:
import json
import os

# Define the path to the directory containing JSON files
google_fit_path = "C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit/All Data"

# Function to read JSON files
def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# List JSON files
json_files = [file for file in os.listdir(google_fit_path) if file.endswith('.json')]

# Inspect the first few JSON files
for json_file in json_files[:3]:
    file_path = os.path.join(google_fit_path, json_file)
    data = read_json_file(file_path)
    print(json.dumps(data, indent=4))
    print("\n\n")


{
    "Data Source": "derived:com.google.active_minutes:com.google.android.fit:Xiaomi:Redmi Note 7:9702862f3104aa6b:top_level",
    "Data Points": [
        {
            "fitValue": [
                {
                    "value": {
                        "intVal": 1
                    }
                }
            ],
            "originDataSourceId": "derived:com.google.step_count.delta:com.google.android.gms:estimated_steps",
            "endTimeNanos": 1650459360000000000,
            "dataTypeName": "com.google.active_minutes",
            "startTimeNanos": 1650459300000000000,
            "modifiedTimeMillis": 1650459928839,
            "rawTimestampNanos": 0
        },
        {
            "fitValue": [
                {
                    "value": {
                        "intVal": 1
                    }
                }
            ],
            "originDataSourceId": "derived:com.google.step_count.delta:com.google.android.gms:estimated_steps",
            "endTimeNan

In [34]:
import os
import json
import pandas as pd

# Define the path to the datasets
google_fit_path = "C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit/All Data"

# Function to read JSON files
def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# Read all JSON files from the directory
json_files = [os.path.join(google_fit_path, file) for file in os.listdir(google_fit_path) if file.endswith('.json')]
all_data = []
for file in json_files:
    data = read_json_file(file)
    all_data.extend(data.get('Data Points', []))

# Function to convert JSON data to DataFrame
def json_to_dataframe(data):
    records = []
    for entry in data:
        if 'startTimeNanos' in entry and 'endTimeNanos' in entry:
            record = {
                "start_time": int(entry.get("startTimeNanos")),
                "end_time": int(entry.get("endTimeNanos")),
                "activity_type": entry.get("dataTypeName"),
                "value": None
            }
            if "fitValue" in entry and len(entry["fitValue"]) > 0 and "value" in entry["fitValue"][0]:
                record["value"] = entry["fitValue"][0]["value"].get("intVal")
            records.append(record)
    df = pd.DataFrame(records)
    if not df.empty:
        df['start_time'] = pd.to_datetime(df['start_time'], unit='ns')
        df['end_time'] = pd.to_datetime(df['end_time'], unit='ns')
    return df

# Process data and create DataFrame
df_google_fit = json_to_dataframe(all_data)

# Display the DataFrame
print(df_google_fit.head())

# Check if DataFrame is empty
if df_google_fit.empty:
    print("No valid data found in the JSON files.")
else:
    # Data Preprocessing (Handling missing values, normalization, etc.)
    df_google_fit.dropna(inplace=True)

    # Convert datetime columns to numerical features
    df_google_fit['duration'] = (df_google_fit['end_time'] - df_google_fit['start_time']).dt.total_seconds()
    df_google_fit['start_time'] = df_google_fit['start_time'].astype('int64') // 10**9
    df_google_fit['end_time'] = df_google_fit['end_time'].astype('int64') // 10**9

    # Feature Engineering (if needed)
    X = df_google_fit[['start_time', 'end_time', 'duration', 'value']]
    y = df_google_fit['activity_type']

    # Train-Test Split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model Training
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Model Evaluation
    y_pred = model.predict(X_test)
    from sklearn.metrics import classification_report
    print(classification_report(y_test, y_pred))

    # Save the model
    import joblib
    joblib.dump(model, 'google_fit_model.pkl')


           start_time            end_time              activity_type  value
0 2022-04-20 12:55:00 2022-04-20 12:56:00  com.google.active_minutes    1.0
1 2022-04-20 12:56:00 2022-04-20 12:57:00  com.google.active_minutes    1.0
2 2022-04-20 12:57:00 2022-04-20 12:58:00  com.google.active_minutes    1.0
3 2022-04-20 12:59:00 2022-04-20 13:00:00  com.google.active_minutes    1.0
4 2022-04-20 13:00:00 2022-04-20 13:01:00  com.google.active_minutes    1.0
                                  precision    recall  f1-score   support

       com.google.active_minutes       0.99      0.97      0.98     13017
     com.google.activity.segment       0.99      0.99      0.99     17103
com.google.step_count.cumulative       1.00      0.99      1.00      2794
     com.google.step_count.delta       1.00      1.00      1.00    135471

                        accuracy                           1.00    168385
                       macro avg       0.99      0.99      0.99    168385
                    weig

In [36]:
import os
import json
import pandas as pd

# Define the path to the datasets
google_fit_path = "C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit/All Data"

# Function to read JSON files
def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# Read all JSON files from the directory
json_files = [os.path.join(google_fit_path, file) for file in os.listdir(google_fit_path) if file.endswith('.json')]
all_data = []
for file in json_files:
    data = read_json_file(file)
    all_data.extend(data.get('Data Points', []))

# Function to convert JSON data to DataFrame
def json_to_dataframe(data):
    records = []
    for entry in data:
        if 'startTimeNanos' in entry and 'endTimeNanos' in entry:
            record = {
                "start_time": int(entry.get("startTimeNanos")),
                "end_time": int(entry.get("endTimeNanos")),
                "activity_type": entry.get("dataTypeName"),
                "value": None
            }
            if "fitValue" in entry and len(entry["fitValue"]) > 0 and "value" in entry["fitValue"][0]:
                record["value"] = entry["fitValue"][0]["value"].get("intVal")
            records.append(record)
    df = pd.DataFrame(records)
    if not df.empty:
        df['start_time'] = pd.to_datetime(df['start_time'], unit='ns')
        df['end_time'] = pd.to_datetime(df['end_time'], unit='ns')
    return df

# Process data and create DataFrame
df_google_fit = json_to_dataframe(all_data)

# Display the DataFrame
print(df_google_fit.head())

# Check if DataFrame is empty
if df_google_fit.empty:
    print("No valid data found in the JSON files.")
else:
    # Data Preprocessing (Handling missing values, normalization, etc.)
    df_google_fit.dropna(inplace=True)

    # Convert datetime columns to numerical features
    df_google_fit['duration'] = (df_google_fit['end_time'] - df_google_fit['start_time']).dt.total_seconds()
    df_google_fit['start_time'] = df_google_fit['start_time'].astype('int64') // 10**9
    df_google_fit['end_time'] = df_google_fit['end_time'].astype('int64') // 10**9

    # Feature Engineering (if needed)
    X = df_google_fit[['start_time', 'end_time', 'duration', 'value']]
    y = df_google_fit['activity_type']

    # Train-Test Split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model Training
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Model Evaluation
    y_pred = model.predict(X_test)
    from sklearn.metrics import classification_report
    print(classification_report(y_test, y_pred))

    # Save the model
    import joblib
    joblib.dump(model, 'google_fit_model.pkl')


           start_time            end_time              activity_type  value
0 2022-04-20 12:55:00 2022-04-20 12:56:00  com.google.active_minutes    1.0
1 2022-04-20 12:56:00 2022-04-20 12:57:00  com.google.active_minutes    1.0
2 2022-04-20 12:57:00 2022-04-20 12:58:00  com.google.active_minutes    1.0
3 2022-04-20 12:59:00 2022-04-20 13:00:00  com.google.active_minutes    1.0
4 2022-04-20 13:00:00 2022-04-20 13:01:00  com.google.active_minutes    1.0
                                  precision    recall  f1-score   support

       com.google.active_minutes       0.99      0.97      0.98     13017
     com.google.activity.segment       0.99      0.99      0.99     17103
com.google.step_count.cumulative       1.00      0.99      1.00      2794
     com.google.step_count.delta       1.00      1.00      1.00    135471

                        accuracy                           1.00    168385
                       macro avg       0.99      0.99      0.99    168385
                    weig

In [2]:
!pip install fpdf


  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40723 sha256=e01a818605d093ee9352a868e517109ec6965f6fe1c49109174d8cf4aeac5792
  Stored in directory: c:\users\pavan\appdata\local\pip\cache\wheels\44\35\8b\86ce00cec7e4d13c5f189680ae0fa82f919bedc066c2cddae9
Successfully built fpdf


In [3]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from fpdf import FPDF
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# Define the path to the datasets
google_fit_path = "C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit/All Data"

# Function to read JSON files
def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# Read all JSON files from the directory
json_files = [os.path.join(google_fit_path, file) for file in os.listdir(google_fit_path) if file.endswith('.json')]
all_data = []
for file in json_files:
    data = read_json_file(file)
    all_data.extend(data.get('Data Points', []))

# Function to convert JSON data to DataFrame
def json_to_dataframe(data):
    records = []
    for entry in data:
        if 'startTimeNanos' in entry and 'endTimeNanos' in entry:
            record = {
                "start_time": int(entry.get("startTimeNanos")),
                "end_time": int(entry.get("endTimeNanos")),
                "activity_type": entry.get("dataTypeName"),
                "value": None
            }
            if "fitValue" in entry and len(entry["fitValue"]) > 0 and "value" in entry["fitValue"][0]:
                record["value"] = entry["fitValue"][0]["value"].get("intVal")
            records.append(record)
    df = pd.DataFrame(records)
    if not df.empty:
        df['start_time'] = pd.to_datetime(df['start_time'], unit='ns')
        df['end_time'] = pd.to_datetime(df['end_time'], unit='ns')
    return df

# Process data and create DataFrame
df_google_fit = json_to_dataframe(all_data)

# Display the DataFrame
print(df_google_fit.head())

# Check if DataFrame is empty
if df_google_fit.empty:
    print("No valid data found in the JSON files.")
else:
    # Data Preprocessing (Handling missing values, normalization, etc.)
    df_google_fit.dropna(inplace=True)

    # Convert datetime columns to numerical features
    df_google_fit['duration'] = (df_google_fit['end_time'] - df_google_fit['start_time']).dt.total_seconds()
    df_google_fit['start_time'] = df_google_fit['start_time'].astype('int64') // 10**9
    df_google_fit['end_time'] = df_google_fit['end_time'].astype('int64') // 10**9

    # Feature Engineering (if needed)
    X = df_google_fit[['start_time', 'end_time', 'duration', 'value']]
    y = df_google_fit['activity_type']

    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model Training
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Model Evaluation
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    print(classification_report(y_test, y_pred))

    # Save the model
    joblib.dump(model, 'google_fit_model.pkl')

    # Generate PDF Report
    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'FitLifeBot Fitness Report', 0, 1, 'C')

        def chapter_title(self, title):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, title, 0, 1, 'L')
            self.ln(10)

        def chapter_body(self, body):
            self.set_font('Arial', '', 12)
            self.multi_cell(0, 10, body)
            self.ln()

        def add_image(self, image_path, x, y, w, h):
            self.image(image_path, x, y, w, h)

    # Create instance of FPDF class
    pdf = PDF()
    pdf.add_page()

    # Add chapter titles and body
    pdf.chapter_title('User Performance Report')
    body = f"User performance based on Google Fit data. Activity types, durations, and counts are analyzed."
    pdf.chapter_body(body)

    # Add classification report
    pdf.chapter_title('Classification Report')
    report_text = json.dumps(report, indent=4)
    pdf.chapter_body(report_text)

    # Save PDF
    pdf_output_path = 'user_fitness_report.pdf'
    pdf.output(pdf_output_path)
    print(f"Report saved as {pdf_output_path}")

    # Add more details or visualizations as needed

# Ensure the code works and outputs the necessary files correctly.


C:\Users\pavan\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


           start_time            end_time              activity_type  value
0 2022-04-20 12:55:00 2022-04-20 12:56:00  com.google.active_minutes    1.0
1 2022-04-20 12:56:00 2022-04-20 12:57:00  com.google.active_minutes    1.0
2 2022-04-20 12:57:00 2022-04-20 12:58:00  com.google.active_minutes    1.0
3 2022-04-20 12:59:00 2022-04-20 13:00:00  com.google.active_minutes    1.0
4 2022-04-20 13:00:00 2022-04-20 13:01:00  com.google.active_minutes    1.0
                                  precision    recall  f1-score   support

       com.google.active_minutes       0.99      0.97      0.98     13017
     com.google.activity.segment       0.99      0.99      0.99     17103
com.google.step_count.cumulative       1.00      0.99      1.00      2794
     com.google.step_count.delta       1.00      1.00      1.00    135471

                        accuracy                           1.00    168385
                       macro avg       0.99      0.99      0.99    168385
                    weig

In [1]:
import os
import json
import pandas as pd
import numpy as np
from fpdf import FPDF
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Define the path to the datasets
google_fit_path = "C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit/All Data"

# Function to read JSON files
def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# Read all JSON files from the directory
json_files = [os.path.join(google_fit_path, file) for file in os.listdir(google_fit_path) if file.endswith('.json')]
all_data = []
for file in json_files:
    data = read_json_file(file)
    all_data.extend(data.get('Data Points', []))

# Function to convert JSON data to DataFrame
def json_to_dataframe(data):
    records = []
    for entry in data:
        if 'startTimeNanos' in entry and 'endTimeNanos' in entry:
            record = {
                "start_time": int(entry.get("startTimeNanos")),
                "end_time": int(entry.get("endTimeNanos")),
                "activity_type": entry.get("dataTypeName"),
                "value": None
            }
            if "fitValue" in entry and len(entry["fitValue"]) > 0 and "value" in entry["fitValue"][0]:
                record["value"] = entry["fitValue"][0]["value"].get("intVal")
            records.append(record)
    df = pd.DataFrame(records)
    if not df.empty:
        df['start_time'] = pd.to_datetime(df['start_time'], unit='ns')
        df['end_time'] = pd.to_datetime(df['end_time'], unit='ns')
    return df

# Process data and create DataFrame
df_google_fit = json_to_dataframe(all_data)

# Display the DataFrame
print(df_google_fit.head())

# Check if DataFrame is empty
if df_google_fit.empty:
    print("No valid data found in the JSON files.")
else:
    # Data Preprocessing (Handling missing values, normalization, etc.)
    df_google_fit.dropna(inplace=True)

    # Convert datetime columns to numerical features
    df_google_fit['duration'] = (df_google_fit['end_time'] - df_google_fit['start_time']).dt.total_seconds()
    df_google_fit['start_time'] = df_google_fit['start_time'].astype('int64') // 10**9
    df_google_fit['end_time'] = df_google_fit['end_time'].astype('int64') // 10**9

    # Introduce some noise into the dataset
    np.random.seed(42)
    noise = np.random.normal(0, 0.1, df_google_fit['value'].shape)
    df_google_fit['value'] = df_google_fit['value'] + noise

    # Feature Engineering (if needed)
    X = df_google_fit[['start_time', 'end_time', 'duration', 'value']]
    y = df_google_fit['activity_type']

    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model Training
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Model Evaluation
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred)
    print(report)

    # Save the model
    import joblib
    joblib.dump(model, 'google_fit_model.pkl')

    # Generate PDF Report
    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'User Fitness Report', 0, 1, 'C')
        
        def chapter_title(self, title):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, title, 0, 1, 'L')
            self.ln(5)
        
        def chapter_body(self, body):
            self.set_font('Arial', '', 12)
            self.multi_cell(0, 10, body)
            self.ln()
    
    pdf = PDF()
    pdf.add_page()
    
    # Add user activity analysis
    activity_summary = df_google_fit['activity_type'].value_counts()
    total_activities = len(df_google_fit)
    active_minutes = df_google_fit[df_google_fit['activity_type'] == 'com.google.active_minutes']['duration'].sum()
    
    activity_analysis = f"""
    Total activities recorded: {total_activities}
    Total active minutes: {active_minutes / 60:.2f} hours
    Activity Summary:
    {activity_summary.to_string()}
    """
    
    pdf.chapter_title('User Activity Analysis')
    pdf.chapter_body(activity_analysis)
    
    # Add model evaluation
    pdf.chapter_title('Model Evaluation')
    pdf.chapter_body(report)
    
    # Save PDF
    pdf_output_path = 'C:/Users/pavan/OneDrive/Documents/MS Project/user_fitness_report.pdf'
    pdf.output(pdf_output_path)
    print(f"Report saved as {pdf_output_path}")


C:\Users\pavan\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


           start_time            end_time              activity_type  value
0 2022-04-20 12:55:00 2022-04-20 12:56:00  com.google.active_minutes    1.0
1 2022-04-20 12:56:00 2022-04-20 12:57:00  com.google.active_minutes    1.0
2 2022-04-20 12:57:00 2022-04-20 12:58:00  com.google.active_minutes    1.0
3 2022-04-20 12:59:00 2022-04-20 13:00:00  com.google.active_minutes    1.0
4 2022-04-20 13:00:00 2022-04-20 13:01:00  com.google.active_minutes    1.0
                                  precision    recall  f1-score   support

       com.google.active_minutes       0.94      0.94      0.94     13017
     com.google.activity.segment       0.99      0.96      0.98     17103
com.google.step_count.cumulative       1.00      0.99      1.00      2794
     com.google.step_count.delta       0.99      0.99      0.99    135471

                        accuracy                           0.99    168385
                       macro avg       0.98      0.97      0.98    168385
                    weig

In [40]:
import os
import json
import pandas as pd
import numpy as np
from fpdf import FPDF
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib
import matplotlib.pyplot as plt

# Define the path to the datasets
google_fit_path = "C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit/All Data"

# Function to read JSON files
def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# Read all JSON files from the directory
json_files = [os.path.join(google_fit_path, file) for file in os.listdir(google_fit_path) if file.endswith('.json')]
all_data = []
for file in json_files:
    data = read_json_file(file)
    all_data.extend(data.get('Data Points', []))

# Function to convert JSON data to DataFrame
def json_to_dataframe(data):
    records = []
    for entry in data:
        if 'startTimeNanos' in entry and 'endTimeNanos' in entry:
            record = {
                "start_time": int(entry.get("startTimeNanos")),
                "end_time": int(entry.get("endTimeNanos")),
                "activity_type": entry.get("dataTypeName"),
                "value": None
            }
            if "fitValue" in entry and len(entry["fitValue"]) > 0 and "value" in entry["fitValue"][0]:
                record["value"] = entry["fitValue"][0]["value"].get("intVal")
            records.append(record)
    df = pd.DataFrame(records)
    if not df.empty:
        df['start_time'] = pd.to_datetime(df['start_time'], unit='ns')
        df['end_time'] = pd.to_datetime(df['end_time'], unit='ns')
    return df

# Process data and create DataFrame
df_google_fit = json_to_dataframe(all_data)

# Display the DataFrame
print(df_google_fit.head())

# Check if DataFrame is empty
if df_google_fit.empty:
    print("No valid data found in the JSON files.")
else:
    # Data Preprocessing (Handling missing values, normalization, etc.)
    df_google_fit.dropna(inplace=True)

    # Ensure datetime columns are correctly parsed
    df_google_fit['start_time'] = pd.to_datetime(df_google_fit['start_time'])
    df_google_fit['end_time'] = pd.to_datetime(df_google_fit['end_time'])

    # Convert datetime columns to numerical features
    df_google_fit['duration'] = (df_google_fit['end_time'] - df_google_fit['start_time']).dt.total_seconds()
    df_google_fit['start_time_epoch'] = df_google_fit['start_time'].astype('int64') // 10**9
    df_google_fit['end_time_epoch'] = df_google_fit['end_time'].astype('int64') // 10**9

    # Feature Engineering
    df_google_fit['day_of_week'] = df_google_fit['start_time'].dt.dayofweek
    df_google_fit['hour_of_day'] = df_google_fit['start_time'].dt.hour

    # Extracting additional relevant features
    def extract_step_count(entry):
        # Logic to extract step count from the data
        return np.random.randint(0, 10000)  # Placeholder value

    def extract_heart_rate(entry):
        # Logic to extract heart rate from the data
        return np.random.randint(60, 100)  # Placeholder value

    df_google_fit['step_count'] = df_google_fit.apply(lambda x: extract_step_count(x), axis=1)
    df_google_fit['heart_rate'] = df_google_fit.apply(lambda x: extract_heart_rate(x), axis=1)

    # Feature selection
    X = df_google_fit[['start_time_epoch', 'end_time_epoch', 'duration', 'value', 'day_of_week', 'hour_of_day', 'step_count', 'heart_rate']]
    y = df_google_fit['activity_type']

    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create a pipeline with scaling and ensemble classifier
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', VotingClassifier(estimators=[
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
            ('gbm', GradientBoostingClassifier(n_estimators=100, random_state=42)),
            ('nn', MLPClassifier(random_state=42))
        ], voting='soft'))
    ])

    # Cross-Validation
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5)
    print(f"Ensemble Cross-validation scores: {cv_scores}")
    print(f"Mean CV score: {np.mean(cv_scores)}")

    # Model Training
    pipeline.fit(X_train, y_train)

    # Model Evaluation
    y_pred = pipeline.predict(X_test)
    report = classification_report(y_test, y_pred)
    print(report)

    # Save the model
    joblib.dump(pipeline, 'google_fit_model.pkl')

    # Generate PDF Report
    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'User Fitness Report', 0, 1, 'C')
        
        def chapter_title(self, title):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, title, 0, 1, 'L')
            self.ln(5)
        
        def chapter_body(self, body):
            self.set_font('Arial', '', 12)
            self.multi_cell(0, 10, body)
            self.ln()

        def add_chart(self, chart_path):
            self.image(chart_path, x=10, y=None, w=180)

    # Create visualizations
    activity_summary = df_google_fit['activity_type'].value_counts()
    activity_summary.plot(kind='bar')
    plt.title('Activity Summary')
    plt.xlabel('Activity Type')
    plt.ylabel('Count')
    chart_path = 'activity_summary.png'
    plt.savefig(chart_path)
    plt.close()

    # Create PDF report
    pdf = PDF()
    pdf.add_page()
    
    # Add user activity analysis
    total_activities = len(df_google_fit)
    active_minutes = df_google_fit[df_google_fit['activity_type'] == 'com.google.active_minutes']['duration'].sum()
    
    activity_analysis = f"""
    This document provides an analysis of the Google Fit dataset passed through an Ensemble Classifier. 
    The following are the results obtained from the dataset analysis:

    Total activities recorded: {total_activities}
    Total active minutes: {active_minutes / 60:.2f} hours
    Activity Summary:
    {activity_summary.to_string()}
    """
    
    pdf.chapter_title('User Activity Analysis')
    pdf.chapter_body(activity_analysis)
    pdf.add_chart(chart_path)
    
    # Add model evaluation
    pdf.chapter_title('Model Evaluation')
    pdf.chapter_body(report)
    
    # Save PDF
    pdf_output_path = 'C:/Users/pavan/OneDrive/Documents/MS Project/user_fitness_report.pdf'
    pdf.output(pdf_output_path)
    print(f"Report saved as {pdf_output_path}")

           start_time            end_time              activity_type  value
0 2022-04-20 12:55:00 2022-04-20 12:56:00  com.google.active_minutes    1.0
1 2022-04-20 12:56:00 2022-04-20 12:57:00  com.google.active_minutes    1.0
2 2022-04-20 12:57:00 2022-04-20 12:58:00  com.google.active_minutes    1.0
3 2022-04-20 12:59:00 2022-04-20 13:00:00  com.google.active_minutes    1.0
4 2022-04-20 13:00:00 2022-04-20 13:01:00  com.google.active_minutes    1.0


C:\Users\pavan\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\pavan\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\pavan\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\pavan\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\pavan\anaconda3\lib\site-packages\sklearn\neural_network\_multi

Ensemble Cross-validation scores: [0.97827894 0.9782344  0.97839756 0.97847179 0.97783337]
Mean CV score: 0.9782432145124643


C:\Users\pavan\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


                                  precision    recall  f1-score   support

       com.google.active_minutes       0.93      0.92      0.93     13017
     com.google.activity.segment       0.99      0.91      0.95     17103
com.google.step_count.cumulative       1.00      0.98      0.99      2794
     com.google.step_count.delta       0.98      0.99      0.99    135471

                        accuracy                           0.98    168385
                       macro avg       0.98      0.95      0.96    168385
                    weighted avg       0.98      0.98      0.98    168385

Report saved as C:/Users/pavan/OneDrive/Documents/MS Project/user_fitness_report.pdf


In [39]:
import os
import pandas as pd
from fpdf import FPDF

# Define the path to the daily activity metrics CSV file
daily_activity_path = "C:/Users/pavan/OneDrive/Documents/MS Project/Takeout/Fit/Daily activity metrics/Daily activity metrics.csv"

# Load daily activity metrics
df_daily_activity = pd.read_csv(daily_activity_path)

# Convert Date column to datetime
df_daily_activity['Date'] = pd.to_datetime(df_daily_activity['Date'], format='%Y-%m-%d')

# Filter the last three months of data
three_months_ago = pd.Timestamp.now() - pd.DateOffset(months=3)
df_daily_activity = df_daily_activity[df_daily_activity['Date'] >= three_months_ago]

# Select only the required columns
required_columns = [
    'Date', 'Calories (kcal)', 'Distance (m)', 'Average speed (m/s)', 'Max speed (m/s)',
    'Min speed (m/s)', 'Step count', 'Heart Points'
]

df_daily_activity = df_daily_activity[required_columns]

# Data Preprocessing (Handling missing values and rounding numbers)
df_daily_activity.dropna(inplace=True)
df_daily_activity = df_daily_activity.round(2)

# Generate month-by-month analysis
monthly_analysis = df_daily_activity.groupby(df_daily_activity['Date'].dt.to_period('M')).mean().reset_index()
monthly_analysis = monthly_analysis.round(2)

# Shorten column names for the PDF
monthly_analysis.columns = [
    'Date', 'Calories', 'Distance', 'Avg Speed m/s ', ' Max Speed',
    'Min Speed', 'Steps', 'Heart Points'
]

# Generate PDF Report
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'User Fitness Report - Last Three Months', 0, 1, 'C')
    
    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)
    
    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

    def add_table(self, data):
        self.set_font('Arial', '', 10)
        page_width = self.w - 2 * self.l_margin  # Effective page width
        col_width = page_width / len(data.columns)  # Equal column width
        for col in data.columns:
            self.cell(col_width, 10, col, border=1)
        self.ln()
        for i in range(len(data)):
            for j in range(len(data.columns)):
                self.cell(col_width, 10, str(data.iloc[i, j]), border=1)
            self.ln()

# Create PDF report
pdf = PDF()
pdf.add_page()

# Add descriptive statistics
pdf.chapter_title('Descriptive Statistics')
pdf.chapter_body(df_daily_activity.describe().round(2).to_string())

# Summarize key metrics
total_calories = round(df_daily_activity['Calories (kcal)'].sum(), 2)
total_distance = round(df_daily_activity['Distance (m)'].sum(), 2)
total_steps = round(df_daily_activity['Step count'].sum(), 2)
average_speed = round(df_daily_activity['Average speed (m/s)'].mean(), 2)
max_speed = round(df_daily_activity['Max speed (m/s)'].max(), 2)
min_speed = round(df_daily_activity['Min speed (m/s)'].min(), 2)
total_heart_points = round(df_daily_activity['Heart Points'].sum()/10, 2)

summary = f"""
Summary of Key Metrics:
Total Calories Burned: {total_calories}
Total Distance Covered (m): {total_distance}
Avg Speed (m/s): {average_speed}
Max Speed: {max_speed}
Min Speed: {min_speed}
Total Steps: {total_steps}
Total Heart Points: {total_heart_points}
"""

pdf.chapter_title('Summary of Key Metrics')
pdf.chapter_body(summary)

# Add month-by-month analysis
pdf.chapter_title('Monthly Analysis')
pdf.add_table(monthly_analysis)

# Add note for LLM consideration
pdf.chapter_title('Note for LLM')
llm_note = """
This analysis report provides a detailed overview of the user's fitness activities over the last three months. 
The metrics analyzed include Move Minutes count, Calories burned, Distance covered, Heart Points, Average speed, 
Max speed, Min speed, and Step count. The LLM should consider this document as the primary report for providing 
meaningful fitness recommendations based on the user's activity patterns.
"""
pdf.chapter_body(llm_note)

# Save PDF
pdf_output_path = 'C:/Users/pavan/OneDrive/Documents/MS Project/user_fitness_report_monthly4.pdf'
pdf.output(pdf_output_path)
print(f"Report saved as {pdf_output_path}")


Report saved as C:/Users/pavan/OneDrive/Documents/MS Project/user_fitness_report_monthly.pdf
